In [1]:
import pandas as pd 
import numpy as np 
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Introduction:
Based on Global Status Report on Road Safety 2018 presented by World Health Organization (WHO), 1.35 million people are killed on roadways around the world each year and almost 3,700 people are killed globally in road traffic crashes every day. 

To reduce the number of accidents and Car collisions, consequently the number of deaths, we should identify factors helping to accurately predict car accident Severity such as the current weather and the road conditions. Therefore, using this model, the conductor will be alerted to drive more carefully or change his travel to reduce the possibility of getting in an accident.


# Data Understanding:
The dataset used for this project is specified in a CSV file "Data-Collisions.csv". 
The target variable will be "SEVERITYCODE". This attribute corresponds to the severity of a collision based on "WEATHER", "ROADCOND" and "LIGHTCOND".


NameError: name 'body' is not defined

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

df_data_collision = pd.read_csv(body)
df_data_collision.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N
